In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from scikeras.wrappers import KerasClassifier

data = pd.read_csv('/content/drive/MyDrive/Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 3, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)


Epoch 1/3
291/291 - 31s - loss: 0.8189 - accuracy: 0.6465 - 31s/epoch - 106ms/step
Epoch 2/3
291/291 - 25s - loss: 0.6768 - accuracy: 0.7125 - 25s/epoch - 87ms/step
Epoch 3/3
291/291 - 26s - loss: 0.6140 - accuracy: 0.7434 - 26s/epoch - 88ms/step
144/144 - 3s - loss: 0.7474 - accuracy: 0.6793 - 3s/epoch - 18ms/step
0.7474012970924377
0.6793359518051147
['loss', 'accuracy']


In [26]:
model.save('sm1')
test = ["A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump"]
#Seems to be positive sentiment
test = tokenizer.texts_to_sequences(test)
test = pad_sequences(test,28)
import tensorflow as tf
sm1 = tf.keras.models.load_model("sm1")
#You don't seem to need to compile after loading the model
sm1.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
sm1.predict(test)
#[negative, neutral, positive]

1/1 [==============================] - 0s 282ms/step


array([[0.29857984, 0.08784208, 0.6135781 ]], dtype=float32)

In [27]:
from scikeras.wrappers import KerasClassifier
batch_size= [16, 32, 64]
epochs = [1, 10, 20]
p_grid= {'batch_size': batch_size, 'epochs': epochs}
gridmodel = KerasClassifier(model=createmodel, verbose=2)
grid  = GridSearchCV(estimator = gridmodel, param_grid = p_grid)
results = grid.fit(X_train, Y_train)

465/465 - 39s - loss: 0.8314 - accuracy: 0.6439 - 39s/epoch - 84ms/step
117/117 - 1s - 1s/epoch - 12ms/step
465/465 - 39s - loss: 0.8214 - accuracy: 0.6512 - 39s/epoch - 84ms/step
117/117 - 2s - 2s/epoch - 14ms/step
465/465 - 39s - loss: 0.8303 - accuracy: 0.6450 - 39s/epoch - 84ms/step
117/117 - 1s - 1s/epoch - 13ms/step
465/465 - 39s - loss: 0.8310 - accuracy: 0.6432 - 39s/epoch - 83ms/step
117/117 - 1s - 1s/epoch - 13ms/step
465/465 - 38s - loss: 0.8263 - accuracy: 0.6456 - 38s/epoch - 82ms/step
117/117 - 2s - 2s/epoch - 14ms/step
Epoch 1/10
465/465 - 38s - loss: 0.8280 - accuracy: 0.6478 - 38s/epoch - 82ms/step
Epoch 2/10
465/465 - 36s - loss: 0.6798 - accuracy: 0.7105 - 36s/epoch - 77ms/step
Epoch 3/10
465/465 - 35s - loss: 0.5991 - accuracy: 0.7556 - 35s/epoch - 74ms/step
Epoch 4/10
465/465 - 36s - loss: 0.5303 - accuracy: 0.7853 - 36s/epoch - 77ms/step
Epoch 5/10
465/465 - 35s - loss: 0.4721 - accuracy: 0.8099 - 35s/epoch - 75ms/step
Epoch 6/10
465/465 - 36s - loss: 0.4245 - acc

ValueError: ignored